In [2]:
!pip install pandas-datareader
import pandas_datareader.data as pdr

import yfinance as yf
import pandas as pd
import numpy as np


data = yf.download('122870.KQ',start='2017-12-01',end='2020-10-31')
df = data
#df구조 출력
df.info()
print(df.index)

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 710 entries, 2017-12-01 to 2020-10-30
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (Adj Close, 122870.KQ)  710 non-null    float64
 1   (Close, 122870.KQ)      710 non-null    float64
 2   (High, 122870.KQ)       710 non-null    float64
 3   (Low, 122870.KQ)        710 non-null    float64
 4   (Open, 122870.KQ)       710 non-null    float64
 5   (Volume, 122870.KQ)     710 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 38.8 KB
DatetimeIndex(['2017-12-01', '2017-12-04', '2017-12-05', '2017-12-06',
               '2017-12-07', '2017-12-08', '2017-12-11', '2017-12-12',
               '2017-12-13', '2017-12-14',
               ...
               '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22',
               '2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28',
               '2020-10-29', '2020-10-30'],
            

In [3]:
# 인덱스를 풀어내고, 'Ticker'와 'Date'를 일반 열로 변환
df.reset_index(inplace=True)

# df_case의 열을 확인 (Ticker와 Date가 열로 변환되었을 것입니다)
print(df.columns)

# 멀티 인덱스를 풀어 인덱스를 일반 열로 변환
df.reset_index(inplace=True)

# 컬럼 이름을 확인하여 'Ticker' 레벨을 제거
print(df.columns)

# 'Ticker' 열을 제거 (두 번째 레벨인 'Ticker'와 관련된 모든 열 제거)
df.columns = df.columns.droplevel(1)  # 'Ticker' 레벨 제거

# 이제 'Date' 열만 인덱스로 설정
df.set_index('Date', inplace=True)

# 결과 확인
print(df.head())



MultiIndex([(     'Date',          ''),
            ('Adj Close', '122870.KQ'),
            (    'Close', '122870.KQ'),
            (     'High', '122870.KQ'),
            (      'Low', '122870.KQ'),
            (     'Open', '122870.KQ'),
            (   'Volume', '122870.KQ')],
           names=['Price', 'Ticker'])
MultiIndex([(    'index',          ''),
            (     'Date',          ''),
            ('Adj Close', '122870.KQ'),
            (    'Close', '122870.KQ'),
            (     'High', '122870.KQ'),
            (      'Low', '122870.KQ'),
            (     'Open', '122870.KQ'),
            (   'Volume', '122870.KQ')],
           names=['Price', 'Ticker'])
Price       index     Adj Close    Close     High      Low     Open  Volume
Date                                                                       
2017-12-01      0  30120.072266  31000.0  31050.0  30450.0  30850.0   76450
2017-12-04      1  29634.265625  30500.0  31200.0  30300.0  31000.0  101695
2017-12-05      2 

In [4]:
from datetime import datetime
file_path = "/content/yg_issue.txt"

#utf-8은 유니코드 변환방식
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 데이터 분리 및 날짜 포맷 변경
data = []
for line in lines:
    line= line.strip()
    date, issue = line.split(" ", 1)  # 날짜와 이슈를 t스페이스 기준으로 분리
    date_a = datetime.strptime(date, "%y.%m.%d").strftime("%Y-%m-%d")  # 날짜 형식 변환
    data.append([date_a, issue])


df_case = pd.DataFrame(data, columns=["Date", "Issue"])

df_case.shape


(61, 2)

In [5]:
df_case['Date'] = pd.to_datetime(df_case['Date'])

print(df_case['Date'].head())

print(df_case.index)
#주가df와 issue df를 붙여야함, merge이용
#장날을 기준으로 이슈를 left merge하면 중요한 이슈들이 빠지는 경우가 생김
#여기서 조건없이 outer 병합을 해두고 이슈를 기준으로 당일 포함 사건 발생 이후 3일로 df를 축소할 것임.
df_analysis = df.merge(df_case.set_index('Date'), left_index=True, right_index=True, how='outer')


df_analysis.head(50)

0   2017-12-18
1   2017-12-31
2   2018-01-01
3   2018-01-25
4   2018-01-31
Name: Date, dtype: datetime64[ns]
RangeIndex(start=0, stop=61, step=1)


,index,Adj Close,Close,High,Low,Open,Volume,Issue
Date,,,,,,,,
2017-12-01,0.0,30120.072266,31000.0,31050.0,30450.0,30850.0,76450.0,NaN
2017-12-04,1.0,29634.265625,30500.0,31200.0,30300.0,31000.0,101695.0,NaN
2017-12-05,2.0,29731.427734,30600.0,30750.0,30050.0,30350.0,81720.0,NaN
2017-12-06,3.0,29245.621094,30100.0,30700.0,30050.0,30700.0,94550.0,NaN
2017-12-07,4.0,28808.392578,29650.0,30300.0,29300.0,30050.0,148490.0,NaN
2017-12-08,5.0,28759.814453,29600.0,30050.0,29500.0,29500.0,76761.0,NaN
2017-12-11,6.0,29877.169922,30750.0,30750.0,29600.0,29600.0,112793.0,NaN
2017-12-12,7.0,28711.230469,29550.0,30800.0,29450.0,30750.0,134828.0,NaN
2017-12-13,8.0,29148.460938,30000.0,30150.0,29550.0,29750.0,109800.0,NaN


In [6]:
df_analysis.drop(columns="index",inplace=True)
df_analysis.head(10)


,Adj Close,Close,High,Low,Open,Volume,Issue
Date,,,,,,,
2017-12-01,30120.072266,31000.0,31050.0,30450.0,30850.0,76450.0,NaN
2017-12-04,29634.265625,30500.0,31200.0,30300.0,31000.0,101695.0,NaN
2017-12-05,29731.427734,30600.0,30750.0,30050.0,30350.0,81720.0,NaN
2017-12-06,29245.621094,30100.0,30700.0,30050.0,30700.0,94550.0,NaN
2017-12-07,28808.392578,29650.0,30300.0,29300.0,30050.0,148490.0,NaN
2017-12-08,28759.814453,29600.0,30050.0,29500.0,29500.0,76761.0,NaN
2017-12-11,29877.169922,30750.0,30750.0,29600.0,29600.0,112793.0,NaN
2017-12-12,28711.230469,29550.0,30800.0,29450.0,30750.0,134828.0,NaN
2017-12-13,29148.460938,30000.0,30150.0,29550.0,29750.0,109800.0,NaN


In [7]:
df_analysis.index

DatetimeIndex(['2017-12-01', '2017-12-04', '2017-12-05', '2017-12-06',
               '2017-12-07', '2017-12-08', '2017-12-11', '2017-12-12',
               '2017-12-13', '2017-12-14',
               ...
               '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22',
               '2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28',
               '2020-10-29', '2020-10-30'],
              dtype='datetime64[ns]', name='Date', length=723, freq=None)

In [8]:
# 1. Issue 열에서 NaN이 아닌 행의 인덱스를 찾기
valid_indices = df_analysis[df_analysis['Issue'].notna()].index

# 2. 해당 행을 포함한 당일과 이후 2일 (총 3일)
indices_to_keep = []

for date in valid_indices:
    # 'date'가 이미 Timestamp 형식이므로 날짜 계산을 바로 수행
    start_date = date  # 당일
    end_date = date + pd.Timedelta(days=2)  # 이후 2일

    # 범위에 맞는 날짜들 인덱스 찾기
    date_range = df_analysis[(df_analysis.index >= start_date) & (df_analysis.index <= end_date)].index
    indices_to_keep.extend(date_range)

# 3. 선택된 인덱스를 제외한 나머지 행 삭제
df_filtered = df_analysis.loc[indices_to_keep]

# 결과 출력
df_filtered



,Adj Close,Close,High,Low,Open,Volume,Issue
Date,,,,,,,
2017-12-18,29439.945312,30300.0,31100.0,29700.0,29850.0,261948.0,태양 결혼 발표
2017-12-19,28905.556641,29750.0,30300.0,29500.0,30300.0,116712.0,NaN
2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,빅뱅 입대 전 4인 콘서트 Last Dance
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,지드래곤-이주연 열애설
2018-01-02,27929.253906,28600.0,29000.0,28500.0,28900.0,82068.0,NaN
...,...,...,...,...,...,...,...
2020-08-28,48271.132812,49000.0,52500.0,47600.0,50900.0,1298641.0,블랙핑크 싱글 Ice Cream 발매
2020-09-08,56743.207031,57600.0,58500.0,54200.0,54700.0,896782.0,트레져 싱글 2집 컴백
2020-09-09,56841.722656,57700.0,60500.0,56200.0,57700.0,821051.0,NaN


In [9]:

!pip install transformers

In [10]:

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import re
import os
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification

In [11]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_data.csv")

('finance_data.csv', <http.client.HTTPMessage at 0x78bba570bfa0>)

In [12]:
data = pd.read_csv('finance_data.csv')
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 4846


In [13]:
data

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,neutral,Rinkuskiai's beer sales fell by 6.5 per cent t...,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,negative,Net sales of the Paper segment decreased to EU...,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [14]:
#감성분석을 실시한 데이터를 결국 회귀분석에 활용할 것이기 때문에 dummy variable 활용을 위해 0,1,2로 처리할 것
data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
data[:5]
#replace이용하여 중립,긍정,부정을 0,1,2로 대체

<ipython-input-14-ae77eccb669c>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])


,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [15]:
del data['sentence']
#해당 data라는 df는 영문과 번역된 한글문을 가지고 있는데, 우리는 한국 뉴스로 train하는것이 목표이기 때문에 영문 문장은 뺀다

In [16]:
data.info()

data[:5]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   labels        4846 non-null   int64 
 1   kor_sentence  4846 non-null   object
dtypes: int64(1), object(1)
memory usage: 75.8+ KB


,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [17]:
print('결측값 여부 :',data.isnull().values.any())
#결측값이 있는지 확인

결측값 여부 : False


In [18]:
#한국어 문장 전처리 시 겹치는 문장이 있다면 제거하고 고유값만 남기도록, 그런데 파인튜닝 하려고 보니 고윳값만 남겼을 때 인덱싱 에러가..
print('kor_sentence 열의 유니크한 값 :',data['kor_sentence'].nunique())

kor_sentence 열의 유니크한 값 : 4827


In [19]:
duplicate = data[data.duplicated()]
duplicate

,labels,kor_sentence
1099,0,이 발표 내용에 대한 책임은 전적으로 발행자에게 있습니다.
1394,0,"핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서..."
1416,0,"이 보고서는 블랙 앤 데커, 피스카스, 피스카스 브랜드, 후스크바르나 아웃도어 프로..."
2396,0,알스트롬의 주가는 나스닥 OMX 헬싱키에서 인용되고 있다.
2567,0,SSH 통신 보안 코퍼레이션은 핀란드 헬싱키에 본사를 두고 있다.
2889,0,재정적인 세부사항은 공개되지 않았다.
2890,0,재정적인 세부사항은 공개되지 않았다.
2892,0,금융 조건은 공개되지 않았다.
3050,0,재정적인 세부사항은 제공되지 않았다.
3051,0,재정적인 세부사항은 공개되지 않았다.


In [20]:
#중복 제거 후 제거된 상태 유지
data.drop_duplicates(subset=['kor_sentence'], inplace=True)

In [21]:
print(f'중립 = {round(data["labels"].value_counts()[0]/len(data) * 100,2)}%')
print(f'긍정 = {round(data["labels"].value_counts()[1]/len(data) * 100,2)}%')
print(f'부정 = {round(data["labels"].value_counts()[2]/len(data) * 100,2)}%')
# 훈련 데이터셋에서의 0,1,2 비율 확인

중립 = 59.27%
긍정 = 28.22%
부정 = 12.51%


In [22]:
X_data = data['kor_sentence']
y_data = data['labels']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)
print('train sample num :', len(X_train))
print('test sample num :', len(X_test))
#split 할 때 가장 일반적이면서도 4800개 가량 존재하는 dataset을 분해할 때의 황금비율인 0.2로 테스트값 분해

train sample num : 3861
test sample num : 966


In [23]:
#train data의 0,1,2비율
print(f'중립 = {round(y_train.value_counts()[0]/len(y_train) * 100,3)}%')
print(f'긍정 = {round(y_train.value_counts()[1]/len(y_train) * 100,3)}%')
print(f'부정 = {round(y_train.value_counts()[2]/len(y_train) * 100,3)}%')

#test data의 0,1,2비율
print(f'중립 = {round(y_test.value_counts()[0]/len(y_test) * 100,3)}%')
print(f'긍정 = {round(y_test.value_counts()[1]/len(y_test) * 100,3)}%')
print(f'부정 = {round(y_test.value_counts()[2]/len(y_test) * 100,3)}%')

중립 = 59.285%
긍정 = 28.205%
부정 = 12.51%
중립 = 59.213%
긍정 = 28.261%
부정 = 12.526%


In [24]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd


In [25]:
{
  "bos_token": "[CLS]", # 시퀀스의 시작
  "cls_token": "[CLS]", # 문장의 시작
  "eos_token": "[SEP]", # 문장의 끝
  "mask_token": "[MASK]", # 마스킹
  "pad_token": "[PAD]", # 입력 시퀀스 길이 맞추기
  "sep_token": "[SEP]", # 문장의 끝
  "unk_token": "[UNK]" # 알 수 없는 토큰
} #토큰 정의

{'bos_token': '[CLS]',
 'cls_token': '[CLS]',
 'eos_token': '[SEP]',
 'mask_token': '[MASK]',
 'pad_token': '[PAD]',
 'sep_token': '[SEP]',
 'unk_token': '[UNK]'}

In [83]:
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.tokenizer = tokenizer  # 토크나이저 객체를 클래스에 저장
        self.data = data            # 데이터프레임을 클래스에 저장
        self.texts = data['kor_sentence']  # 'kor_sentence' 열을 추출하여 텍스트 저장
        self.targets = data['labels']  # 'labels' 열을 추출하여 레이블 저장
        self.max_len = max_len      # 모델 입력 시퀀스의 최대 길이 설정


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.texts[index])
        target = self.targets[index]

        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        return {
            'input_ids': input_ids.long(),
            'attention_mask': attention_mask.long(),
            'labels': torch.tensor(target, dtype=torch.long)
        }


kor_sentence    핀에어의 총 교통량은 수익 여객 킬로미터 측면에서 8.7% 감소했다.
labels                                               2
Name: 1798, dtype: object
이 커뮤니케이션과 관련된 모든 투자 또는 투자 활동은 관련자만 이용할 수 있으며 관련자와만 참여할 수 있습니다.
0
kor_sentence    핀에어의 총 교통량은 수익 여객 킬로미터 측면에서 8.7% 감소했다.
labels                                               2
Name: 1798, dtype: object


In [26]:
# 데이터프레임을 'data'로 이미 불러왔다고 가정
# 'ko_sentence'는 텍스트 컬럼이고, 'label'은 감성 레이블 컬럼
# 예시로 데이터셋을 나눈다고 가정
X_data = data['kor_sentence']
y_data = data['labels']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

train_data = pd.DataFrame({'kor_sentence': X_train, 'labels': y_train})
test_data = pd.DataFrame({'kor_sentence': X_test, 'labels': y_test})

class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.texts = dataframe['kor_sentence']  # 'ko_sentence' 컬럼 사용
        self.targets = dataframe['labels'].astype(int)  # 'label'을 정수형으로 변환
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.texts.iloc[index])
        target = self.targets.iloc[index]

        # tokenizer를 이용해 입력 데이터를 처리
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze(0)  # 첫 번째 차원을 제거
        attention_mask = inputs['attention_mask'].squeeze(0)  # 첫 번째 차원을 제거

        # 데이터타입을 torch.long으로 명시
        return {
            'input_ids': input_ids.long(),  # input_ids를 long 타입으로 변환
            'attention_mask': attention_mask.long(),  # attention_mask를 long 타입으로 변환
            'labels': torch.tensor(target, dtype=torch.long)  # labels도 long 타입으로 변환
        }


# 모델과 토크나이저 로드
model_name = "kakaobank/kf-deberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 감성분석은 보통 3개의 레이블(긍정, 부정, 중립)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 훈련을 위한 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="epoch",  # 에폭마다 평가
    save_strategy="epoch"  # 에폭마다 모델 저장
)

# 데이터셋 준비
max_len = 128  # 시퀀스 최대 길이
train_dataset = SentimentDataset(train_data, tokenizer, max_len)
test_dataset = SentimentDataset(test_data, tokenizer, max_len)

# Trainer 객체 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer  # Trainer에서 tokenizer도 전달
)

# 모델 학습 시작
trainer.train()

# 모델 저장
trainer.save_model('./sentiment_model')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/746M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at kakaobank/kf-deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.393050
2,No log,0.510080
3,0.521600,0.412855
4,0.521600,0.512477
5,0.204100,0.659423
